# 0.0 Imports

In [21]:
import pandas as pd
import inflection

## 0.1 Helper Functionts

## 0.2 Loading Data

In [16]:
df_sales_raw = pd.read_csv( 'C:/Users/felip/ds_em_producao/ds_em_producao/datasets/train.csv' , low_memory = False)
df_store_raw = pd.read_csv( 'C:/Users/felip/ds_em_producao/ds_em_producao/datasets/store.csv' , low_memory = False)


# merging datasets

df_raw = pd.merge(df_sales_raw , df_store_raw , how = 'left' , on = 'Store')

# 1.0 Descrição dos Dados

In [17]:
df1 = df_raw.copy()

## 1.1 Rename Columns

In [22]:
cols_old = ['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo','StateHoliday', 'SchoolHoliday', 'StoreType', 'Assortment',
            'CompetitionDistance', 'CompetitionOpenSinceMonth','CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek',
            'Promo2SinceYear', 'PromoInterval']

snakecase = lambda x: inflection.underscore(x)

cols_new = list(map(snakecase , cols_old))

# rename

df1.columns = cols_new

## 1.2 Data Dimentions

In [25]:
print('Number of Rows: {}'.format(df1.shape[0]))
print('Number of Columns: {}'.format(df1.shape[1]))

Number of Rows: 1017209
Number of Columns: 18


## 1.3 Data Types

In [28]:
df1['date'] = pd.to_datetime(df1['date'])

df1.dtypes

store                                    int64
day_of_week                              int64
date                            datetime64[ns]
sales                                    int64
customers                                int64
open                                     int64
promo                                    int64
state_holiday                           object
school_holiday                           int64
store_type                              object
assortment                              object
competition_distance                   float64
competition_open_since_month           float64
competition_open_since_year            float64
promo2                                   int64
promo2_since_week                      float64
promo2_since_year                      float64
promo_interval                          object
dtype: object

## 1.4 Check NA

In [29]:
df1.isna().sum()

store                                0
day_of_week                          0
date                                 0
sales                                0
customers                            0
open                                 0
promo                                0
state_holiday                        0
school_holiday                       0
store_type                           0
assortment                           0
competition_distance              2642
competition_open_since_month    323348
competition_open_since_year     323348
promo2                               0
promo2_since_week               508031
promo2_since_year               508031
promo_interval                  508031
dtype: int64

## 1.2 Imports

## 1.2 Imports

## 1.2 Imports

## 1.2 Imports